In [ ]:
import os
from dotenv import load_dotenv
import json
import random
import matplotlib.pyplot as plt
import numpy as np
import asyncio

from benchmarklib import BenchmarkDatabase
from clique import CliqueProblem, CliqueTrial, CliqueOracleTrial
from benchmarklib.compilers import CompileType, XAGCompiler

from qiskit_ibm_runtime import QiskitRuntimeService

import logging
from typing import Iterable, List, Tuple, Dict, Any, Union, Optional
import qiskit
from qiskit.providers import Backend
from qiskit import QuantumCircuit, transpile
import random

from tweedledum.bool_function_compiler import circuit_input, QuantumCircuitFunction
from tweedledum import BitVec

from benchmarklib import CompileType, BenchmarkDatabase
from benchmarklib import BatchQueue
from benchmarklib.compilers import SynthesisCompiler

from sqlalchemy import func, select

In [ ]:
# Load Qiskit
from qiskit_ibm_runtime import QiskitRuntimeService, Batch
from dotenv import load_dotenv, find_dotenv
import os

service = QiskitRuntimeService()
backend = service.backend(name="ibm_rensselaer")

In [ ]:
benchmark_db = BenchmarkDatabase("clique.db", CliqueProblem, CliqueTrial)

### Check for synthesis mistakes

In [ ]:
from benchmarklib.algorithms.grover import verify_oracle
from sqlalchemy.orm import joinedload
for trial in benchmark_db.query(
    select(CliqueTrial).where(CliqueTrial._circuit_pretranspile_qpy != None)
    .order_by(func.random())
    .limit(100)
    .options(joinedload(CliqueTrial.problem))
):
    oracle = trial.circuit
    if not verify_oracle(oracle, trial.problem):
        print(f"Oracle verification failed for trial ID {trial.problem.graph} compiled with {trial.compiler_name}")

### Validating QMPC Compiler

In [ ]:
from benchmarklib.pipeline import PipelineCompiler
from benchmarklib.pipeline.synthesis import QuantumMPC
from benchmarklib.algorithms.grover import verify_oracle

compiler_qmpc = PipelineCompiler(
    synthesizer = QuantumMPC(),
    # Transpilation is an optional step, but passing a backend & transpile options will do it automatically
    #steps = [QiskitTranspile(backend=backend, optimization_level=3)]
    steps = [], 
    backend = backend, 
    transpile_options = {"optimization_level": 3}
)

In [ ]:
graph = "0111001101"
p = benchmark_db.find_problem_instances(graph=graph, limit=1)[0]
compilation_result = compiler_qmpc.compile(p, clique_size=max(p.nodes//2, 2))

In [ ]:
compilation_result.artifacts["truth_table"]

In [ ]:
oracle = compilation_result.synthesis_circuit



In [ ]:
oracle2 = p.oracle("XAG", clique_size=max(p.nodes//2, 2))

In [ ]:
verify_oracle(oracle, p)

In [ ]:
from sqlalchemy import func, select
problems = benchmark_db.query(
    select(CliqueProblem).where(CliqueProblem.nodes <= 4).order_by(func.random()).limit(10)
)
for p in problems:
    print(f"Verifying problem {p.graph}")
    compilation_result = compiler_qmpc.compile(p, clique_size=max(p.nodes//2, 2))
    if not verify_oracle(compilation_result.synthesis_circuit, p):
        print(f"Verification failed for problem {p.graph}")
        print(compilation_result.synthesis_circuit.draw())

In [ ]:
for n in range(2, 21):
    count = benchmark_db.query(
        select(func.count(CliqueProblem.id))
        .where(CliqueProblem.nodes == n)
    )
    print(f"n={n}: {count}")

In [ ]:
for n in range(2, 21):
    count = benchmark_db.query(
        select(func.count(CliqueProblem.id))
        .select_from(CliqueTrial)
        .join(CliqueTrial.problem)
        .where(CliqueProblem.nodes == n)
    )
    print(f"n={n}: {count}")

In [ ]:
count = 0
for n in range(2, 21):
    num_edges =  n * (n - 1) // 2
    count += 2**num_edges

print(count)